In [1]:
from nemo_text_processing.text_normalization.normalize import Normalizer

In [2]:
normaliser_sv = Normalizer(input_case='cased', lang='sv', cache_dir="/tmp/nemo_cache_sv", overwrite_cache=False, post_process=True)

Created /tmp/nemo_cache_sv/sv_tn_True_deterministic_cased__tokenize.far
Created /tmp/nemo_cache_sv/sv_tn_True_deterministic_verbalizer.far


In [3]:
from pathlib import Path
APIFILES = Path("/home/joregan/rdapi_subset_files")
OUTFILES = Path("/home/joregan/rdapi_subset_norm")

In [4]:
import json

In [5]:
with open(str(APIFILES / "H101CU11")) as apifile:
    rdfile = json.load(apifile)

In [ ]:
for viddata in rdfile['videodata']:
    for speaker in viddata['speakers']:
        print(speaker['anftext'])